# Class for reviews

In [1]:
import time
import random
from bs4 import BeautifulSoup

class product_reviews(object):
    
    def __init__(self,driver,link):
        self.driver = driver
        self.reviews_for_product(link)
        
    def reviews_for_product(self,link):
        self.driver.get(link)
        reviewlink = self.driver.find_element_by_id("dp-summary-see-all-reviews")
        reviewlink.click()
        ###
        time.sleep(5+random.uniform(0,1))
        ###
        reviewlink = self.driver.find_element_by_xpath('//*[@id="cm_cr-review_list"]/div[1]/span[5]/a')
        reviewlink.click()

        reviews = []
        while True:
            time.sleep(5+random.uniform(0,2))
            page = self.driver.page_source
            soup = BeautifulSoup(page,"lxml")
            reviews += self.review_from_page(soup)

            if soup.find("li",class_="a-last").find("a",href=True): 
                #nextpage = self.driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[8]')
                nextpage = self.driver.find_element_by_class_name('a-last')
                nextpage.click()
            else:
                break

        self.reviews = reviews

    def review_from_page(self,soup):
        items = soup.find_all("div",class_="a-section review")
        reviews = []
        for item in items:
            reviews.append(self.review(item))
        return reviews

    def review(self,review_item_soup):
        review_dict = {}
        review_dict["star"] = review_item_soup.find("span",class_="a-icon-alt").text.split()[0]
        review_dict["review_title"] = review_item_soup.find("a",class_="a-size-base a-link-normal review-title a-color-base a-text-bold").text
        review_dict["author"] = review_item_soup.find("a",class_="a-size-base a-link-normal author").text
        review_dict["date"] = review_item_soup.find("span",class_="a-size-base a-color-secondary review-date").text
        review_dict["review_text"] = review_item_soup.find("span",class_="a-size-base review-text").text
        return review_dict

# Class for products

In [2]:
import time
import random
from bs4 import BeautifulSoup

class product_information(object):
    
    def __init__(self,driver,link):
        self.driver = driver
        self.link = link
        self.driver.get(self.link)
        
        self.soup = BeautifulSoup(self.driver.page_source,"lxml")
        self.product_information()

    def product_information(self):
        information = {}
        information["title"] = self.title()
        information["price"] = self.price()
        information["shipping"] = self.shipping()
        information["availability"] = self.availability()
        information["images"] = self.images()
        information["details"] = self.details()
        
        information["reviews"] = product_reviews(self.driver,self.link).reviews
        
        self.information = information

    def title(self):
        try:
            return self.soup.find("span",id="productTitle").text.strip(' \n')
        except:
            return "NA"
        
    def images(self):
        try:
            images = str(self.soup.find("div",class_="imgTagWrapper"))
            return [image for image in images.split('"') if 'https://' in image]
        except:
            return "NA"
    
    def price(self):
        try:
            return self.soup.find("span",id="priceblock_ourprice").text
        except:
            return "NA"
        
    
    def shipping(self):
        try:
            return self.soup.find("span",id="price-shipping-message").text.strip(' \n')
        except:
            return "NA"
    
    def availability(self):
        try:
            tmplist = self.soup.find("div",id="availability-brief").text.split("\n")
            return " ".join([s.strip() for s in tmplist if s.strip()])
        except:
            return "NA"
        
    def details(self):
        #try:
        details = self.soup.find_all("div",class_="attrG")
        detail_list = []
        for detail in details:
            for td in detail.find_all("td"):
                if td.text != u'\xa0':
                    tmp = " ".join([el for el in td.text.split('\n') if el])
                    detail_list.append(tmp.split(";")[-1])
        return {detail_list[2*i]:detail_list[2*i+1] for i in range(len(detail_list)/2)}
        #except:
        #    return "NA"

## main

In [3]:
f = open("links.dat","r")
lines = f.readlines()
f.close()

In [4]:
from selenium import webdriver
import time
import os
import json

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [41]:
# prod = product_information(driver,link)
# data = prod.information
# with open(data["details"]['ASIN']+'.json', 'w') as fp:
#     json.dump(data, fp)

In [67]:
# page = driver.page_source
# soup = BeautifulSoup(page,'lxml')
# for el in soup.find_all("div",class_="attrG"):
#     print el.text

In [71]:
import sys

L = len(lines)
for i in range(25,L):
    link = lines[i]
    time.sleep(3+random.uniform(0,3))
    prod = product_information(driver,link)
    data = prod.information
    #sys.stdout.write("L"+str(i))
    with open(data["details"]['ASIN']+'.json', 'w') as fp:
        json.dump(data, fp)
    sys.stdout.write("L"+str(i))

L15L16L17L18L19L20L21L22L23L24

TimeoutException: Message: timeout
  (Session info: chrome=58.0.3029.110)
  (Driver info: chromedriver=2.29.461585 (0be2cd95f834e9ee7c46bcc7cf405b483f5ae83b),platform=Mac OS X 10.12.4 x86_64)


In [5]:
L = len(lines)
i = 45
while i<L:
    try:
        link = lines[i]
        #print link
        time.sleep(5+random.uniform(0,2))
        prod = product_information(driver,link)
        data = prod.information
        #sys.stdout.write("L"+str(i))
        with open(data["details"]['ASIN']+'.json', 'w') as fp:
            json.dump(data, fp)
        sys.stdout.write("L"+str(i))
        i += 1
    except:
        sys.stdout.write("P")
        time.sleep(1800+random.uniform(0,300))

NameError: name 'sys' is not defined

In [ ]:
https://www.amazon.com/gp/goldbox/ref=sr_deal_ldhttps://www.amazon.com/Summer-Infant-3Dflip-Convenience-Stroller/dp/B00OZJ6MQI/ref=sr_1_25_s_it?s=baby-products&ie=UTF8&qid=1495584358&sr=1-25&keywords=stroller
